In [138]:
#Alexandria Benedict, Assignment 2


import nltk
import matplotlib.pyplot as plt
from nltk.probability import FreqDist
import pandas as pd
from math import log
from sklearn.model_selection import train_test_split
from collections import *
from sklearn.metrics import accuracy_score

## Question 1 

In [205]:
#Take in data
df1 = pd.read_csv('CONcreTEXT_trial_EN.tsv', sep='\t', header=None, engine= 'python')
df2 = pd.read_csv('CONcreTEXT_trial_IT.tsv', sep='\t', header=None, engine= 'python')

# ENG
list1 = []
for s in df1[3][1:]:
    words = nltk.word_tokenize(s)
    list1 += [word.lower() for word in words if word.isalpha()]
    
# IT
list2 = []
for s in df2[3][1:]:
    words = nltk.word_tokenize(s)
    list2 += [word.lower() for word in words if word.isalpha()]

#Split data into training and testing data sets
eTest = []
eTrain = []
iTest = []
iTrain = []
print("English:\n")
print("Original len: " , len(list1))
eTrain, eTest = train_test_split(list1, test_size = .2, train_size = .8, random_state=42)
print("Train size: " , len(eTrain))
print("Test size: " , len(eTest), "\n")
print("Italian:\n")
iTrain, iTest = train_test_split(list2, test_size = .2, train_size = .8, random_state=42)
print("Original len: " , len(list2))
print("Train size: " , len(iTrain))
print("Test size: " , len(iTest))


English:

Original len:  1294
Train size:  1035
Test size:  259 

Italian:

Original len:  1303
Train size:  1042
Test size:  261


### Question 2 
I did this part without using FreqDist to get a better understanding of the process manually. I used FreqDist for the bigrams and trigrams afterwards.

In [207]:
#Get list of characters
eUni = []
for s in eTrain:
    eUni.append(list(s))
iUni = []
for s in iTrain:
    iUni.append(list(s))
    

#Store counts of each character appearing
test = {}
eCount = 0
for i in eUni:
    for c in i:
        eCount += 1
        if c in test:
            test[c] += 1
        else:
            test[c] = 1

test2 = {}
iCount = 0
for i in iUni:
    for c in i:
        iCount += 1
        if c in test2:
            test2[c] += 1
        else:
            test2[c] = 1

# Calculate Frequencies

testProbEng = {}
for i in test:
    testProbEng[i] = test[i]/eCount
    
testProbIt = {}
for i in test2:
    testProbIt[i] = test2[i]/iCount


#PROBABILITIES
#English
testProbabilities = []
for word in eTest:
    wordList = []
    
    wordList.append(word)
    tempChars = list(word)
    eProb = 1
    iProb = 1
    
    for char in tempChars:
        if char in testProbEng:
            eProb *= testProbEng[char]
        else:
            eProb *= 0
        if char in testProbIt:
            iProb *= testProbIt[char]
        else:
            iProb *= 0
        
 
    wordList.append(eProb)
    wordList.append(iProb)
    testProbabilities.append(wordList)

#Get Accuracy
eng_predUni = []
eng_trueUni = []

for i in testProbabilities:
    eng_trueUni.append("e")
    if i[1] > i[2]:
        eng_predUni.append("e")
    elif i[2] > i[1]:
        eng_predUni.append("i")
    else:
        eng_predUni.append("ei")
        
        
engUnigramAccuracy = accuracy_score(eng_trueUni, eng_predUni)
print("Accuracy:\t",engUnigramAccuracy * 100, "%")



Accuracy:	 58.68725868725869 %


## Question 3

Comparing to the unigram character model, the bigram character model had a higher accuracy, totaling to 76.59%. I think that the bigram character model had increased accuracy compared to unigrams because it is more likely for a bigram to contain language-specific pairs of characters rather than the unigram model. For unigrams, there are many  letters which are shared between both English and Italian making it more difficult to distinguish; however, some of the accented characters belong only to Italian. Because of this, with bigrams there is a higher probability of having a pair of letters which contain language-specific characters.

In [179]:
engBigrams = []
itBigrams = []

#Get Bigrams for each training dataset
for word in eTrain:
    temp = list(nltk.bigrams(word))
    engBigrams += temp

for word in iTrain:
    temp = list(nltk.bigrams(word))
    itBigrams += temp
    
engDist = nltk.FreqDist(engBigrams)
itDist = nltk.FreqDist(itBigrams)

#Calculate probabilities

engBigramProb = []
for word in eTest:
    #Skipped words which are shorter than 2 letters since we are doing bigrams
    if len(word) < 2: continue
    tempBigrams = list(nltk.bigrams(word))
    wordList = []
    
    wordList.append(word)
    eProb = 1
    iProb = 1
    
    for gram in tempBigrams:
        if gram in engDist.keys():
            y = engDist[gram]
            x = 0
            for z in engBigrams:
                if z[0] == gram[0]: x += 1
            eProb *= y/x
        else:
            eProb *= 0
        if gram in itDist.keys():
            y = itDist[gram]
            x = 0
            for z in itBigrams:
                if z[0] == gram[0]: x += 1
            iProb *= y/x
        else:
            iProb *= 0
    
    wordList.append(eProb)
    wordList.append(iProb)
    engBigramProb.append(wordList)
    
"""    
print("Word\tEng Prob\tIt Prob")
for i in engBigramProb:
    print(i[0],"\t",i[1],"\t",i[2],"\t")
"""
#Get Accuracy
eng_pred = []
eng_true = []

for i in engBigramProb:
    eng_true.append("e")
    if i[1] > i[2]:
        eng_pred.append("e")
    else:
        eng_pred.append("i")
        
        
engBigramAccuracy = accuracy_score(eng_true, eng_pred)
print("Accuracy:\t",engBigramAccuracy * 100, "%")

Accuracy:	 76.5873015873016 %


## Question 4

When doing the trigram character model, the accuracy decreased a bit from the bigram model to be 62.79%. I think the reason why the trigram model accuracy decreased is because of our training dataset size. Trigram character models are more specific due to checking 3 character pairs rather than 2, so there are more possible outcomes which need to be trained. Since our corpus used to train isn't considerably large, there is a possibility for less outcomes being accounted for, meaning the accuracy would be lower. I think if we had a larger corpus to train, the accuracies for prediction would be higher.

In [209]:
engTrigrams = []
itTrigrams = []


#Splitting training datasets into trigrams
for word in eTrain:
    temp = list(nltk.trigrams(word))
    engTrigrams += temp
    
for word in iTrain:
    temp = list(nltk.trigrams(word))
    itTrigrams += temp

#Get frequencies of trigrams
engTriDist = nltk.FreqDist(engTrigrams)
itTriDist = nltk.FreqDist(itTrigrams)

#Probabilities

engTrigramProb = []
for word in eTest:
    #Since we are doing trigrams, skip words which are less than 3 letters long
    if len(word) < 3: continue
    tempTrigrams = list(nltk.trigrams(word))
    wordList = []
    
    wordList.append(word)
    eProb = 1
    iProb = 1
    
    for gram in tempTrigrams:
        if gram in engTriDist.keys():
            y = engTriDist[gram]
            x = 0
            for z in engTrigrams:
                if z[0] == gram[0] and z[1] == gram[1]: x += 1
            eProb *= y/x
        if gram in itTriDist.keys():
            y = itTriDist[gram]
            x = 0
            for z in itTrigrams:
                if z[0] == gram[0] and z[1] == gram[1]: x += 1
            iProb *= y/x
    #If a probability remains 1, it did not occur
    if eProb is 1: eProb = 0
    if iProb is 1: iProb = 0
    wordList.append(eProb)
    wordList.append(iProb)
    engTrigramProb.append(wordList)
    
  


#Get Accuracy
eng_pred2 = []
eng_true2 = []

for i in engTrigramProb:
    eng_true2.append("e")
    if i[1] > i[2]:
        eng_pred2.append("e")
    elif i[2] > i[1]:
        eng_pred2.append("i")
    else:
        eng_pred2.append("ei")
        
        
engTrigramAccuracy = accuracy_score(eng_true2, eng_pred2)
print("Accuracy:\t",engTrigramAccuracy * 100, "%")

Accuracy:	 62.7906976744186 %
